In [1]:
import torch
from data_processing import load_interactions, create_interaction_matrix, create_adj_matrix
from model import LightGCN
from training import train_lightgcn
from recommend import get_recommendations

def main():
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load and process data
    print("Loading data...")
    likes_df, posts_df = load_interactions()
    interaction_matrix, user_mapping, post_mapping = create_interaction_matrix(likes_df)
    
    # Print matrix dimensions
    print(f"Interaction matrix shape: {interaction_matrix.shape}")
    print(f"Number of users: {len(user_mapping)}")
    print(f"Number of posts: {len(post_mapping)}")
    print(f"Number of interactions: {interaction_matrix.nnz}")
    
    # Create adjacency matrix
    print("Creating adjacency matrix...")
    adj_matrix = create_adj_matrix(interaction_matrix)
    print(f"Adjacency matrix shape: {adj_matrix.size()}")
    
    # Initialize model
    print("Initializing model...")
    model = LightGCN(
        num_users=len(user_mapping),
        num_items=len(post_mapping),
        embedding_dim=32,  # Reduced from 64
        num_layers=2      # Reduced from 3
    )
    
    # Training setup with memory-efficient parameters
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    
    # Train model with smaller batch size
    print("Training model...")
    train_lightgcn(
        model=model,
        optimizer=optimizer,
        train_data=interaction_matrix,
        adj_matrix=adj_matrix,
        epochs=100,
        batch_size=128,  # Smaller batch size
        device=device
    )
    
    # Example: Get recommendations for a user
    print("Getting recommendations...")
    user_id = list(user_mapping.keys())[0]  # Get first user as example
    recommendations = get_recommendations(
        model=model,
        user_id=user_id,
        user_mapping=user_mapping,
        post_mapping=post_mapping,
        adj_matrix=adj_matrix,
        top_k=10,
        device=device
    )
    
    print(f"Top 10 recommendations for user {user_id}:")
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")

if __name__ == "__main__":
    main()

Loading data...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Loaded 3618997 interactions and 15 unique posts before May 2023
Interaction matrix shape: (32836, 3618981)
Number of users: 32836
Number of posts: 3618981
Number of interactions: 3618997
Creating adjacency matrix...
Adjacency matrix shape: torch.Size([3651817, 3651817])
Initializing model...


c:\Users\Aleck\DSC180\Bluesky-GraphRec\lightgcn\data_processing.py:82: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:643.)
  adj = torch.sparse.FloatTensor(


Training model...
Epoch 0: Loss = 0.6862
Epoch 10: Loss = 1.0408
Epoch 20: Loss = 1.2053
Epoch 30: Loss = 0.9113
Epoch 40: Loss = 0.9513
Epoch 50: Loss = 0.6722
Epoch 60: Loss = 1.1442
Epoch 70: Loss = 0.6080
Epoch 80: Loss = 0.8622
Epoch 90: Loss = 0.8989
Getting recommendations...
Top 10 recommendations for user did:plc:42kmtf65uqs765coei7bimwx:
1. 3jthfslydbi2w
2. 3jpiqcsurkc2g
3. 3jumn5l5bw42m
4. 3jsuomeu5oc2m
5. 3jmsqkdts6i2e
6. 3ju2j42srud2a
7. 3jnxnpuaiu22c
8. 3jqhl2wsh722r
9. 3jui7a36kts2w
10. 3jnb53kf3yu2f
